# **Exemples d'applications de la notion de graphe orienté**
Cette section du cours te permet de découvrir des applications de la notion de graphe orienté pour des problèmes concrets.


## **Exemple 1** Organisation de métiers
Sur le chantier d'un bâtiment plusieurs corps de métier doivent intervenir. Il y a des impératifs d'enchainement des différents métiers. Par exemple l'électricien veut intervenir après le peintre et le peintre veut intervenir après le plaquiste et le maçon. Pour l'organisateur du chantier il faut voir si les exigences de chacun sont compatibles et proposer une solution simple pour régler les conflits.

| métier | intervient après |
|:- |:- |
|électricien | peintre, carreleur, couvreur |
|peintre | maçon ,plaquiste |
|menuisier | maçon, carreleur, couvreur |
|maçon | couvreur |
|couvreur | charpentier |
|charpentier | maçon |
|carreleur | maçon, peintre, plombier |
|plombier| maçon , electricien |
|plaquiste | maçon, couvreur |
|soliste | tous les autres |

1. Dessine le graphe orienté dont les sommets sont les différents métiers et où un arc $A \rightarrow B$ signifie que $B$ veut intervenir après $A$.
2. Comment voit-on sur le graphe 
 * le métier qui intervient en premier ?
 * le métier qui intervient en dernier ?
 * les conflits ?

---

## **Exemple 2** Le problème des bidons
Dans le film Die Hard 3 (Une journée en enfer), les deux héros, John McClane et Zeus Carver, doivent résoudre l’énigme de Simon Gruber pour arrêter le compte à rebours d’une bombe. Voici l’énigme : « Sur la fontaine, il y a deux bidons : l’un a une contenance de 5 gallons, l’autre de 3 gallons. Remplissez l’un des bidons de 4 gallons d’eau exactement et placez-le sur la balance. La minuterie s’arrêtera. Soyez extrêmement précis : un gramme de plus ou de moins et c’est l’explosion ! ». Les nerfs de John McClane sont alors mis à rude épreuve pour trouver une solution. Il commence par remarquer très justement qu’on ne peut pas remplir le bidon de 3 gallons avec 4 gallons d’eau. Il faut donc trouver le moyen de mettre exactement 4 gallons d’eau dans le bidon de 5 gallons. Dans la scène du film (que vous pouvez consulter en français sur
https://www.youtube.com/watch?v=pmk2mNf9iqE), John commence par donner une première idée peu convaincante puisqu’elle termine par la nécessité de remplir le bidon de 3 gallons au tiers, ce qu’on ne sait faire précisément... Le film propose ensuite une
solution très partielle, coupée au montage. Appliquons donc des méthodes de graphes
orientés pour retrouver la meilleure solution possible que les héros appliquent pour
s’en sortir.
Une configuration du système correspond au volume d’eau contenu dans chacun des deux bidons. On peut donc représenter une telle configuration par une paire $(a , b)$ où a est le volume d’eau contenu dans le bidon de 5 gallons et b le volume d’eau contenu dans le bidon de 3 gallons, avec $0 \le a \le 5$ et $0 \le b \le 3$. Les actions élémentaires possibles du système sont de remplir un des deux bidons (qu’il soit initialement vide ou pas), vider un des deux bidons (qu’il soit initialement plein ou pas) et transférer le contenu d’un des bidons dans l’autre jusqu’à ce que ce dernier soit plein.

1. Ecrire en Python les fonctions qui correspondent à chacune des actions.
 Chaque fonction a pour argument une configuration qui est un tuple $(a,b)$ et doit    renvoyer la configuration obtenue après l'action.

```python
remplir_A (config) 
remplir_B (config) 
vider_A(config)
vider_B(config)
transfere_A_B(config)
transfere_B_A(config)
```
 Ecris aussi une fonction 
 ```python
est_terminale(config) 
```
qui renvoie `True`si la configuration est de la forme $(4,b)$

In [47]:
# code à effacer version élèves
def remplir_A(config) :
    a,b = config
    return 5, b

def remplir_B(config) :
    a,b = config
    return a, 3

def vider_A(config) :
    a,b = config
    return 0, b

def vider_B(config) :
    a,b = config
    return a, 0

def transfere_A_B(config) :
    a,b = config
    place_dans_b = 3-b
    transfert = min (a, place_dans_b)
    return a-transfert, b+transfert

def transfere_B_A(config) :
    a,b = config
    place_dans_a = 5-a
    transfert = min (b, place_dans_a)
    return a+transfert, b-transfert

def est_terminale(config) : 
    a,b = config
    return a == 4

**Il est important que tes fonctions soient correctes. Voici un jeu de tests, à toi de le compléter pour tester chaque fonction.**

In [14]:
initial = (0,0)
config1 = (5,0)
config2 = (0,3)
config3 = (2,3)

# test de remplir_A 
assert( remplir_A(initial) == (5,0) )
assert( remplir_A(config1) == (5,0) )
assert( remplir_A(config2) == (5,3) )

# test de remplir_B
assert( remplir_B(initial) == (0,3) )

# test de vider_A 
assert( vider_A(config1) == (0,0) )

# test de vider_B
assert( vider_B(config1) == (5,0) )

#test de transfere_A_B
assert( transfere_A_B(config1) == (2,3) 
       
#test de transfere_B_A 


 Les configurations possibles du système en partant de (0,0) s'organisent selon un graphe orienté dont voici le début (on a écrit sur les arètes les actions effectuées)

<img src="images/graphe_bidons.png" alt="drawing" width="200"/>

 Nous allons écrire un algorithme qui construit le graphe de toutes les configurations possibles, en conservant l'information de l'action sur les arètes.
 
2. **Recherche des successeurs**
 La liste des actions possibles est

In [15]:
actions = [remplir_A, remplir_B, vider_A, vider_B, transfere_A_B, transfere_B_A]

Le graphe sera stocké par liste d'adjacence selon un **dictionnaire** dont les clés sont des tuples (configurations) et chaque valeur est un dictionnaire { tuple : str , tuple : str, ...} des configurations atteignables.

Exemple, en partant de la configuration $(0,0)$ on a

$(0,0) \stackrel{\text{vider_A }}{\longrightarrow} (0,0) $ 

$(0,0) \stackrel{\text{remplir_A }}{\longrightarrow} (5,0)$

$ (0,0) \stackrel{\text{remplir_B }}{\longrightarrow} (0,3)$.

On doit avoir dans le dictionnaire :
```python
{ (0,0) : {(0,0): "vider_A",
           (5,0): "remplir_A",
           (0,3): "remplir_B" } ,
 # 
}
```

**Ecrire** une fonction `successeurs(config)` qui renvoie le dictionnaire des configurations atteignables depuis config.

`successeurs((0,0))` 
 doit renvoyer

`{(5, 0): 'remplir_A', (0, 3): 'remplir_B', (0, 0): 'vider_A'}`

Aide : si action est un élément de la liste `actions` on a le code 
```python 
s = action(config)     # s est le successeur de config avec l'action 
nom = action.__name__  # name est une (str) contenant le nom de la fonction


In [24]:
def successeurs(config) :
    # code à effacer version élèves
    dict_succ = dict()
    for action in actions :
        succ = action(config) 
        if succ not in dict_succ : dict_succ[succ] = action.__name__
    return dict_succ

N'oublie pas de tester cette fonction

In [23]:
successeurs((0,0))

{(5, 0): 'remplir_A', (0, 3): 'remplir_B', (0, 0): 'vider_A'}

In [25]:
successeurs((1,1))

{(5, 1): 'remplir_A',
 (1, 3): 'remplir_B',
 (0, 1): 'vider_A',
 (1, 0): 'vider_B',
 (0, 2): 'transfere_A_B',
 (2, 0): 'transfere_B_A'}

3. **Construction du graphe**
Sachant que l'on a $0 \le a \ le 5 $ et $0 \le b \le 3 $, combien de configurations possibles y a t-il ?

 Ecris un script Python qui renvoie la liste de toutes les configurations.

In [27]:
# code à supprimer dans la version élèves
configurations = [ (a,b) for a in range(6) for b in range(4)]
configurations

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (5, 0),
 (5, 1),
 (5, 2),
 (5, 3)]

Ecris ensuite un script qui construit le dictionnaire { config : dictionnaire des configurations atteignables } 

In [29]:
# code  à supprimer dans la version élèves
graphe = { config : successeurs(config) for config in configurations} 
graphe

{(0, 0): {(5, 0): 'remplir_A', (0, 3): 'remplir_B', (0, 0): 'vider_A'},
 (0, 1): {(5, 1): 'remplir_A',
  (0, 3): 'remplir_B',
  (0, 1): 'vider_A',
  (0, 0): 'vider_B',
  (1, 0): 'transfere_B_A'},
 (0, 2): {(5, 2): 'remplir_A',
  (0, 3): 'remplir_B',
  (0, 2): 'vider_A',
  (0, 0): 'vider_B',
  (2, 0): 'transfere_B_A'},
 (0, 3): {(5, 3): 'remplir_A',
  (0, 3): 'remplir_B',
  (0, 0): 'vider_B',
  (3, 0): 'transfere_B_A'},
 (1, 0): {(5, 0): 'remplir_A',
  (1, 3): 'remplir_B',
  (0, 0): 'vider_A',
  (1, 0): 'vider_B',
  (0, 1): 'transfere_A_B'},
 (1, 1): {(5, 1): 'remplir_A',
  (1, 3): 'remplir_B',
  (0, 1): 'vider_A',
  (1, 0): 'vider_B',
  (0, 2): 'transfere_A_B',
  (2, 0): 'transfere_B_A'},
 (1, 2): {(5, 2): 'remplir_A',
  (1, 3): 'remplir_B',
  (0, 2): 'vider_A',
  (1, 0): 'vider_B',
  (0, 3): 'transfere_A_B',
  (3, 0): 'transfere_B_A'},
 (1, 3): {(5, 3): 'remplir_A',
  (1, 3): 'remplir_B',
  (0, 3): 'vider_A',
  (1, 0): 'vider_B',
  (4, 0): 'transfere_B_A'},
 (2, 0): {(5, 0): 'remplir_

4. **Parcours du graphe** 
Ecris un algorithme qui parcourt le graphe en largeur en partant de la configuration initiale (0,0) jusqu'à arriver à une configuration de la forme (4,b). 
Lorsque l'algorithme rencontre une telle configuration il renvoie True.
Si le parcours se termine sans avoir rencontré une configuration de la forme (4,b) l'algorithme renvoie False.


Au fur et à mesure de l'exploration, l'algorithme complète un dictionnaire des predecesseurs permettant 

In [56]:
def explore_graphe(g,s):
    """ explore le graphe g à partir du sommet s à la recherche d'un sommet terminal
        renvoie le sommet trouve ou None si l'exploration ne trouve pas  """
    visites = [s]    # liste des sommets visités
    file = [s] # File FIFO des sommets à traiter 
    while file != [] :
        # code à supprimer en version élèves
        sommet = file.pop() 
        for successeur in g[sommet] :
            if successeur not in visites : 
                visites.append(successeur)
                if est_terminale(successeur): return successeur
                file.insert(0,successeur)
                
    

In [58]:
explore_graphe(graphe, (0,0))

(4, 3)

Modifie ce code pour qu'il construise un dictionnaire des predecesseurs de la forme 

`{ tuple1 : (tuple2 ,str_action) }`   tel que l'on ait $ tuple_1 \stackrel{\text{str_action }}{\longrightarrow} tuple_2 $

et qu'il renvoie aussi ce dictionnaire.

In [61]:
def explore_graphe2(g,s):
    """ explore le graphe g à partir du sommet s à la recherche d'un sommet terminal
        renvoie le sommet trouve ou None si l'exploration ne trouve pas  """
   
    visites = [s]    # liste des sommets visités
    file = [s] # File FIFO des sommets à traiter 
    predecesseurs = { s : None} # dictionnaire des predecesseurs
    while file != [] :
        # code à supprimer en version élèves
        sommet = file.pop() 
        for successeur in g[sommet] :
            if successeur not in visites : 
                visites.append(successeur)
                predecesseurs[successeur] = (sommet ,g[sommet][successeur] )
                if est_terminale(successeur) : return successeur , predecesseurs
                file.insert(0,successeur)
                
    return None, predecesseurs

In [68]:
final , predecesseurs = explore_graphe2(graphe, (0,0))
print(final) 
predecesseurs

(4, 3)


{(0, 0): None,
 (5, 0): ((0, 0), 'remplir_A'),
 (0, 3): ((0, 0), 'remplir_B'),
 (5, 3): ((5, 0), 'remplir_B'),
 (2, 3): ((5, 0), 'transfere_A_B'),
 (3, 0): ((0, 3), 'transfere_B_A'),
 (2, 0): ((2, 3), 'vider_B'),
 (3, 3): ((3, 0), 'remplir_B'),
 (0, 2): ((2, 0), 'transfere_A_B'),
 (5, 1): ((3, 3), 'transfere_B_A'),
 (5, 2): ((0, 2), 'remplir_A'),
 (0, 1): ((5, 1), 'vider_A'),
 (4, 3): ((5, 2), 'transfere_A_B')}

 5. **Reconstruction du chemin**
 
 L'algorithme précédent a trouvé un sommet terminal a partir de la configuration $(0,0)$, renvoie ce sommet ainsi qu'un dictionnaire contenant les prédécesseurs des sommets visités. 
 Ecris une fonction Python `chemin(predecesseurs, initial, final)` qui renvoie la liste des actions à effectuer pour aller de initial à final.

In [74]:
def chemin(predecesseurs, initial, final) :
    liste_actions = []
    sommet = final
    # code à supprimer en version élèves
    while predecesseurs[sommet] != None :
        pred , action = predecesseurs[sommet]
        liste_actions.insert(0, action) 
        sommet = pred
    return liste_actions

In [75]:
chemin(predecesseurs,(0,0) , (4,3))

['remplir_A',
 'transfere_A_B',
 'vider_B',
 'transfere_A_B',
 'remplir_A',
 'transfere_A_B']